# Model 1: Baseline Autoencoder (Textbook Version)

This notebook implements the **baseline autoencoder** from *Generative Deep Learning, 2nd Edition* (Chapter 3).

**Architecture:**
- **Encoder:** 784 → 256 → 128 → 64 → **64** (latent)
- **Decoder:** **64** (latent) → 64 → 128 → 256 → 784
- **Latent Dimension:** 64
- **Optimizer:** RMSprop (lr=0.001)
- **Loss:** Mean Squared Error
- **Training:** 20 epochs, batch_size=128

**Reference:** [autoencoder.ipynb](https://github.com/davidADSP/Generative_Deep_Learning_2nd_Edition/blob/main/notebooks/03_vae/01_autoencoder/autoencoder.ipynb)

In [1]:
# Import required libraries
import os
import numpy as np
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import MeanSquaredError
from PIL import Image

## 1. Load and Preprocess Fashion-MNIST Dataset

In [2]:
# Load Fashion-MNIST dataset
print("Loading Fashion-MNIST dataset...")
(x_train, _), (x_test, _) = fashion_mnist.load_data()

# Normalize pixel values to [0, 1]
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Flatten input images for dense autoencoder
x_train_flat = x_train.reshape((-1, 28 * 28))
x_test_flat = x_test.reshape((-1, 28 * 28))

print(f"Training samples: {x_train_flat.shape[0]}")
print(f"Test samples: {x_test_flat.shape[0]}")

Loading Fashion-MNIST dataset...
Training samples: 60000
Test samples: 10000


## 2. Build Autoencoder Architecture

In [3]:
# Build the autoencoder architecture (exactly as textbook)
input_dim = 28 * 28
latent_dim = 64  # same latent dimension as in the notebook

# Encoder
encoder_input = Input(shape=(input_dim,), name='encoder_input')
x = Dense(256, activation='relu', name='enc_dense_1')(encoder_input)
x = Dense(128, activation='relu', name='enc_dense_2')(x)
x = Dense(64, activation='relu', name='enc_dense_3')(x)
latent = Dense(latent_dim, activation='relu', name='latent')(x)

encoder = Model(inputs=encoder_input, outputs=latent, name='encoder')

# Decoder
decoder_input = Input(shape=(latent_dim,), name='decoder_input')
x = Dense(64, activation='relu', name='dec_dense_1')(decoder_input)
x = Dense(128, activation='relu', name='dec_dense_2')(x)
x = Dense(256, activation='relu', name='dec_dense_3')(x)
decoder_output = Dense(input_dim, activation='sigmoid', name='decoder_output')(x)

decoder = Model(inputs=decoder_input, outputs=decoder_output, name='decoder')

# Autoencoder: encoder + decoder
ae_input = encoder_input
ae_output = decoder(encoder(ae_input))
autoencoder = Model(inputs=ae_input, outputs=ae_output, name='autoencoder')

## 3. Compile Model

In [4]:
# Compile model
optimizer = RMSprop(learning_rate=0.001)  # same as notebook
loss_fn = MeanSquaredError()  # reconstruction loss
autoencoder.compile(optimizer=optimizer, loss=loss_fn)

## 4. Model Summaries

In [5]:
# Print model summaries
print("\n" + "=" * 50)
print("Encoder Summary:")
print("=" * 50)
encoder.summary()

print("\n" + "=" * 50)
print("Decoder Summary:")
print("=" * 50)
decoder.summary()

print("\n" + "=" * 50)
print("Autoencoder Summary:")
print("=" * 50)
autoencoder.summary()


Encoder Summary:


Model: "encoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)      │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dense_1 (Dense)             │ (None, 256)            │       200,960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dense_2 (Dense)             │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dense_3 (Dense)             │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent (Dense)                  │ (None, 64)             │         4,160 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 246,272 (962.00 KB)

 Trainable params: 246,272 (962.00 KB)

...

Model: "autoencoder"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ encoder_input (InputLayer)      │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ encoder (Functional)            │ (None, 64)             │       246,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ decoder (Functional)            │ (None, 784)            │       246,992 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 493,264 (1.88 MB)

 Trainable params: 493,264 (1.88 MB)

 Non-trainable params: 0 (0.00 B)

## 5. Train the Model

In [6]:
# Train the model
epochs = 20  # as in the notebook
batch_size = 128  # as in the notebook

print("\n" + "=" * 50)
print("Training Autoencoder...")
print("=" * 50)

history = autoencoder.fit(
    x_train_flat, x_train_flat,
    epochs=epochs,
    batch_size=batch_size,
    shuffle=True,
    validation_data=(x_test_flat, x_test_flat),
    verbose=1
)


Training Autoencoder...
Epoch 1/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.0663 - val_loss: 0.0446
Epoch 2/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0405 - val_loss: 0.0362
Epoch 3/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0337 - val_loss: 0.0313
Epoch 4/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0295 - val_loss: 0.0269
Epoch 5/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0272 - val_loss: 0.0259
Epoch 6/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0258 - val_loss: 0.0255
Epoch 7/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0246 - val_loss: 0.0241
Epoch 8/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0237 - val_loss: 0.0231
Epoch 9/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0228 - val_loss: 0.0220
Epoch 10/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0221 - val_loss: 0.0213
Epoch 11/20
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0216 - val_loss: 0.0214
Epoch 12/20
469/469 ━━━━━━━

## 6. Training Loss per Epoch

In [7]:
# Print training loss per epoch
print("\n" + "=" * 50)
print("Training Loss per Epoch:")
print("=" * 50)
for epoch, loss in enumerate(history.history['loss'], start=1):
    print(f"Epoch {epoch}: {loss:.6f}")

# Final training loss
final_train_loss = history.history['loss'][-1]


Training Loss per Epoch:
Epoch 1: 0.066325
Epoch 2: 0.040515
Epoch 3: 0.033678
Epoch 4: 0.029501
Epoch 5: 0.027184
Epoch 6: 0.025775
Epoch 7: 0.024646
Epoch 8: 0.023727
Epoch 9: 0.022837
Epoch 10: 0.022127
Epoch 11: 0.021577
Epoch 12: 0.021059
Epoch 13: 0.020612
Epoch 14: 0.020211
Epoch 15: 0.019836
Epoch 16: 0.019522
Epoch 17: 0.019196
Epoch 18: 0.018923
Epoch 19: 0.018670
Epoch 20: 0.018432


## 7. Evaluate on Test Set

In [8]:
# Evaluate on test set
print("\n" + "=" * 50)
print("Evaluating on test set...")
print("=" * 50)
test_loss = autoencoder.evaluate(x_test_flat, x_test_flat, batch_size=batch_size, verbose=1)


Evaluating on test set...
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0185


## 8. Reconstruct Test Images

In [9]:
# Reconstruct test images
print("\n" + "=" * 50)
print("Reconstructing test images...")
print("=" * 50)

# Select exactly 5 test images (indices 0, 1, 2, 3, 4)
n_images = 5
indices = [0, 1, 2, 3, 4]
test_samples = x_test_flat[indices]
reconstructed = autoencoder.predict(test_samples, verbose=0)

# Reshape for saving
originals = test_samples.reshape((n_images, 28, 28))
recos = reconstructed.reshape((n_images, 28, 28))

# Ensure output directory exists
output_dir = "reconstructions"
os.makedirs(output_dir, exist_ok=True)

print("\nSelected test image indices:")
for idx in indices:
    print(f"  Index: {idx}")

# Save images
print("\nSaving images...")
for i in range(n_images):
    orig_path = os.path.join(output_dir, f"original_{i+1}.png")
    reco_path = os.path.join(output_dir, f"reconstructed_{i+1}.png")
    
    # Original image
    img = (originals[i] * 255).astype('uint8')
    Image.fromarray(img, mode='L').save(orig_path)
    
    # Reconstructed image
    img2 = (np.clip(recos[i], 0, 1) * 255).astype('uint8')
    Image.fromarray(img2, mode='L').save(reco_path)
    
    print(f"  Saved: {orig_path}")
    print(f"  Saved: {reco_path}")


Reconstructing test images...

Selected test image indices:
  Index: 0
  Index: 1
  Index: 2
  Index: 3
  Index: 4

Saving images...
  Saved: reconstructions/original_1.png
  Saved: reconstructions/reconstructed_1.png
  Saved: reconstructions/original_2.png
  Saved: reconstructions/reconstructed_2.png
  Saved: reconstructions/original_3.png
  Saved: reconstructions/reconstructed_3.png
  Saved: reconstructions/original_4.png
  Saved: reconstructions/reconstructed_4.png
  Saved: reconstructions/original_5.png
  Saved: reconstructions/reconstructed_5.png


In [10]:
# Print final model outputs
print("\n----------------------------------------------")
print("MODEL 1: BASELINE AUTOENCODER (TEXTBOOK VERSION)")
print("----------------------------------------------")
print(f"Final Training Loss: {final_train_loss:.6f}")
print(f"Final Test Reconstruction Loss: {test_loss:.6f}")


----------------------------------------------
MODEL 1: BASELINE AUTOENCODER (TEXTBOOK VERSION)
----------------------------------------------
Final Training Loss: 0.018432
Final Test Reconstruction Loss: 0.018549
